# Import

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten
from keras import Model, Sequential, layers, regularizers, optimizers
from keras.callbacks import EarlyStopping
from overcome_tomorrow.utils.data import *
from overcome_tomorrow.ml_logic.preprocess import *
from overcome_tomorrow.ml_logic.model import *
from tqdm import tqdm
pd.set_option('display.max_columns', None)
from sklearn import set_config; set_config(display='diagram')

2024-03-24 18:33:54.835768: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-24 18:33:54.965233: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load model

In [2]:
garmin_data = merge_all_data("../raw_data/Wellness/", "../raw_data/Fitness/", "../raw_data/Aggregator/")
activities = pd.read_csv("../raw_data/activities.csv", parse_dates=["timestamp", "start_time"])

✅ ../raw_data/Wellness/ loaded
✅ Wellness cleaned
✅ ../raw_data/Fitness/ loaded
✅ Fitness cleaned


/home/nico/code/nicblanc/overcome-tomorrow/overcome_tomorrow/utils/data.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_fitness_df["date"] = pd.to_datetime(
/home/nico/code/nicblanc/overcome-tomorrow/overcome_tomorrow/utils/data.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_fitness_df["date"] = pd.to_datetime(clean_fitness_df["date"])
/home/nico/code/nicblanc/overcome-tomorrow/overcome_tomorrow/utils/data.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

✅ ../raw_data/Aggregator/ loaded
✅ Aggregator cleaned
✅ Datasets merged


In [3]:
garmin_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3445 entries, 2019-08-29 to 2024-03-16
Data columns (total 52 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   start_sleep                            3445 non-null   datetime64[ns]
 1   end_sleep                              3445 non-null   datetime64[ns]
 2   deepSleepSeconds                       3440 non-null   float64       
 3   lightSleepSeconds                      3440 non-null   float64       
 4   remSleepSeconds                        3421 non-null   float64       
 5   awakeSleepSeconds                      3440 non-null   float64       
 6   qualityScore                           1902 non-null   float64       
 7   beginTimestamp                         3383 non-null   datetime64[ns]
 8   activityTrainingLoad                   3361 non-null   float64       
 9   activityType                           3383 n

In [4]:
garmin_data.sort_values(by=["beginTimestamp"], inplace=True)
garmin_data.dropna(subset = ["beginTimestamp"], inplace=True)
garmin_data.tail(3)

,start_sleep,end_sleep,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,qualityScore,beginTimestamp,activityTrainingLoad,activityType,aerobicTrainingEffect,aerobicTrainingEffectMessage,anaerobicTrainingEffect,anaerobicTrainingEffectMessage,avgBikeCadence,avgHr,avgPower,avgRunCadence,avgSpeed,calories,caloriesConsumed,distance,duration,maxHr,maxPower,maxRunCadence,maxSpeed,moderateIntensityMinutes,normPower,sportType,trainingEffectLabel,trainingStressScore,vigorousIntensityMinutes,totalKilocalories,activeKilocalories,totalDistanceMeters,wellnessDistanceMeters,highlyActiveSeconds,activeSeconds,floorsAscendedInMeters,floorsDescendedInMeters,restingHeartRate,minAvgHeartRate,maxAvgHeartRate,hydration_sweatLossInML,respiration_avgWakingRespirationValue,respiration_highestRespirationValue,respiration_lowestRespirationValue,bodyBattery_Highest,bodyBattery_Lowest,allDayStress_awake,allDayStress_asleep
calendarDate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-03-15,2024-03-14 20:07:00,2024-03-15 02:47:53,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 05:25:09,144.809784,gravel_cycling,3.5,IMPACTING_TEMPO_22,0.0,NO_ANAEROBIC_BENEFIT_0,77.0,127.0,218.0,NaN,0.8657,8459.65038,NaN,7.261684e+06,8.388064e+06,142.0,418.0,NaN,1.1915,1.0,221.0,CYCLING,TEMPO,147.399994,129.0,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0
2024-03-15,2024-03-14 20:07:00,2024-03-15 02:47:53,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 10:57:38,0.604935,strength_training,0.0,NO_AEROBIC_BENEFIT_18,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,75.0,NaN,NaN,0.0000,79.61038,NaN,0.000000e+00,4.022590e+05,90.0,NaN,NaN,NaN,NaN,NaN,FITNESS_EQUIPMENT,UNKNOWN,NaN,NaN,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0
2024-03-15,2024-03-14 20:07:00,2024-03-15 02:47:53,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 11:14:42,22.125320,running,1.5,RECOVERY_5,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,126.0,319.0,84.0,0.3521,586.60280,NaN,2.337730e+05,6.639260e+05,139.0,442.0,88.0,0.4749,0.0,323.0,RUNNING,RECOVERY,NaN,10.0,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0


In [5]:
activities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3363 entries, 0 to 3362
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype              
---  ------                           --------------  -----              
 0   timestamp                        3363 non-null   datetime64[ns, UTC]
 1   total_anaerobic_training_effect  3358 non-null   float64            
 2   enhanced_avg_speed               3361 non-null   float64            
 3   avg_stroke_distance              332 non-null    float64            
 4   total_training_effect            3358 non-null   float64            
 5   num_active_lengths               317 non-null    float64            
 6   max_running_cadence              714 non-null    float64            
 7   max_heart_rate                   3356 non-null   float64            
 8   sub_sport                        3363 non-null   object             
 9   total_descent                    2392 non-null   float64            
 10  

In [6]:
activities.sort_values(by=["start_time"], inplace=True)
activities.reset_index(drop=True, inplace=True)
activities.tail(3)

,timestamp,total_anaerobic_training_effect,enhanced_avg_speed,avg_stroke_distance,total_training_effect,num_active_lengths,max_running_cadence,max_heart_rate,sub_sport,total_descent,start_time,total_distance,pool_length,178,total_calories,max_cadence,sport,188,avg_step_length,enhanced_max_speed,205,num_lengths,206,207,pool_length_unit,normalized_power,training_load_peak,total_ascent,avg_power,total_strokes,training_stress_score,avg_cadence,avg_heart_rate,max_power,activity_id
3360,2024-03-15 07:45:23+00:00,0.0,8.657,NaN,3.5,NaN,NaN,142.0,gravel_cycling,298.0,2024-03-15 05:25:09+00:00,72616.84,NaN,1371.0,2019.0,104.0,cycling,3.0,NaN,11.915,95.0,NaN,67.0,67.0,NaN,221.0,144.809784,296.0,218.0,10737.0,147.4,77.0,127.0,418.0,nicko64@hotmail.fr_241349530713
3361,2024-03-15 11:05:33+00:00,0.0,0.000,NaN,0.0,NaN,NaN,90.0,strength_training,NaN,2024-03-15 10:57:38+00:00,0.00,NaN,39.0,19.0,NaN,training,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.604935,NaN,NaN,NaN,NaN,NaN,75.0,NaN,nicko64@hotmail.fr_241374800667
3362,2024-03-15 11:26:17+00:00,0.0,3.521,NaN,1.5,NaN,88.0,139.0,generic,13.0,2024-03-15 11:14:42+00:00,2337.73,NaN,184.0,140.0,88.0,running,1.0,1265.6,4.749,95.0,NaN,91.0,91.0,NaN,323.0,22.125320,16.0,319.0,NaN,NaN,84.0,126.0,442.0,nicko64@hotmail.fr_241377639543


# Fit Preprocessors

In [4]:
preproc_garmin_data = create_preproc_garmin_data(garmin_data)
preproc_activity = create_preproc_activity(activities)

In [112]:
preproc_garmin_data

ColumnTransformer(transformers=[('knn_transformer',
                                 Pipeline(steps=[('knn_imputer',
                                                  KNNImputer(n_neighbors=4)),
                                                 ('robust_scaler',
                                                  RobustScaler())]),
                                 Index(['deepSleepSeconds', 'lightSleepSeconds', 'remSleepSeconds',
       'awakeSleepSeconds', 'qualityScore', 'activityTrainingLoad',
       'aerobicTrainingEffect', 'anaerobicTrainingEffect', 'avgBikeCadence',
       'avgHr', 'avgPower...
      dtype='object')),
                                ('pipe_categorical',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one_hot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['sportType', 'trainingEffectLabel']),
                                ('cycle transform', CyclicalFeaturesSleep(),
                                 Index(['start_sleep', 'end_sleep', 'beginTimestamp'], dtype='object'))])

In [113]:
preproc_activity

InvertableColumnTransformer(transformers=[('knn_imputer',
                                           Pipeline(steps=[('knn_imputer',
                                                            KNNImputer(n_neighbors=4)),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['max_heart_rate', 'total_distance',
                                            'total_training_effect',
                                            'training_load_peak', '188', '178',
                                            '206', 'total_descent',
                                            'total_ascent', 'avg_heart_rate',
                                            'total_calories',
                                            'enhanced_avg_speed',
                                            'total_anaerobic...
                                                            SimpleImputer(strategy='most_frequent')),
                                                           ('one_hot',
                                                            OneHotEncoder(drop='if_binary',
                                                                          handle_unknown='ignore',
                                                                          sparse_output=False))]),
                                           ['sport']),
                                          ('cycle_encoder',
                                           CyclicalFeaturesActivity(),
                                           ['timestamp', 'start_time']),
                                          ('imputer 100',
                                           Pipeline(steps=[('simple_imputer',
                                                            SimpleImputer(fill_value=100,
                                                                          strategy='constant')),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['205'])])

In [5]:
preproc_garmin_data.fit(garmin_data)
preproc_activity.fit(activities)

InvertableColumnTransformer(transformers=[('knn_imputer',
                                           Pipeline(steps=[('knn_imputer',
                                                            KNNImputer(n_neighbors=4)),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['enhanced_avg_speed', '207',
                                            'max_heart_rate',
                                            'total_training_effect',
                                            'total_descent',
                                            'total_anaerobic_training_effect',
                                            'total_ascent', '188',
                                            'avg_heart_rate',
                                            'training_load_peak', '178',
                                            'enhanced_max_sp...
                                                            SimpleImputer(strategy='most_frequent')),
                                                           ('one_hot',
                                                            OneHotEncoder(drop='if_binary',
                                                                          handle_unknown='ignore',
                                                                          sparse_output=False))]),
                                           ['sport']),
                                          ('cycle_encoder',
                                           CyclicalFeaturesActivity(),
                                           ['timestamp', 'start_time']),
                                          ('imputer 100',
                                           Pipeline(steps=[('simple_imputer',
                                                            SimpleImputer(fill_value=100,
                                                                          strategy='constant')),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['205'])])

In [115]:
dump(preproc_garmin_data, open("preproc_garmin_data.pkl", "wb"))
dump(preproc_activity, open("preproc_activity.pkl", "wb"))

# Create slide windows

In [11]:
preproc_activity.transform(activities.iloc[[50]])

,knn_imputer__training_load_peak,knn_imputer__avg_heart_rate,knn_imputer__206,knn_imputer__max_heart_rate,knn_imputer__total_anaerobic_training_effect,knn_imputer__total_calories,knn_imputer__188,knn_imputer__total_distance,knn_imputer__enhanced_avg_speed,knn_imputer__total_training_effect,knn_imputer__178,knn_imputer__total_ascent,knn_imputer__enhanced_max_speed,knn_imputer__207,knn_imputer__total_descent,imputer 100__205,cat_encoder__sport_cycling,cat_encoder__sport_fitness_equipment,cat_encoder__sport_generic,cat_encoder__sport_hiking,cat_encoder__sport_kayaking,cat_encoder__sport_rock_climbing,cat_encoder__sport_running,cat_encoder__sport_swimming,cat_encoder__sport_training,cat_encoder__sport_transition,cat_encoder__sport_walking,cycle_encoder__timestamp_sin,cycle_encoder__timestamp_cos,cycle_encoder__start_time_sin,cycle_encoder__start_time_cos
50,-0.37211,-0.585859,0.805755,-0.704545,0.0,-0.614973,-1.0,-0.31803,-0.290368,-0.75,-0.705292,-0.412607,-0.319958,0.780822,-0.340909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.258819,-0.965926,0.258819,-0.965926


In [12]:
days = 30 
avg_activities_per_day = 2 
steps = days * avg_activities_per_day # sliding window size
# Prepare the training data
X_train = []
y_train = []

for i in tqdm(range(steps, activities.shape[0])):
# for i in range(steps, steps + 5):
    activity = activities.iloc[[i]]
    activity_time = activity["start_time"][i].strftime('%Y-%m-%d %H:%M:%S')
    window_df = garmin_data[garmin_data["beginTimestamp"] < activity_time].iloc[i - steps:i]
    X_train.append(preproc_garmin_data.transform(window_df))
    # TODO y_train.append(preproc_activity.transform(activity)[0]) 
    y_train.append(preproc_activity.transform(activity)) 

X_train, y_train = np.array(X_train), np.array(y_train)

print(f"{X_train.shape = } - {y_train.shape = }")

X_train.shape = (3303, 60, 66) - y_train.shape = (3303, 1, 31)


In [22]:
y_train = y_train.reshape((y_train.shape[0], y_train.shape[2])) # To remove, see above

In [23]:
y_train.shape

(3303, 31)

# Create model

In [26]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation
from keras.models import load_model

In [65]:
epochs = 100

model = Sequential()
model.add(LSTM(units=50, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1]))
model.compile(loss="mse", optimizer="adam", metrics=['accuracy'])


In [66]:
model.fit(X_train, y_train, batch_size = 32, epochs = epochs)
model.summary

Epoch 1/100
104/104 [==============================] - 8s 33ms/step - loss: 1.0851 - accuracy: 0.1054
Epoch 2/100
104/104 [==============================] - 3s 27ms/step - loss: 1.0408 - accuracy: 0.1471
Epoch 3/100
104/104 [==============================] - 3s 29ms/step - loss: 1.0209 - accuracy: 0.1614
Epoch 4/100
104/104 [==============================] - 3s 28ms/step - loss: 1.0093 - accuracy: 0.1596
Epoch 5/100
104/104 [==============================] - 3s 29ms/step - loss: 1.0059 - accuracy: 0.1623
Epoch 6/100
104/104 [==============================] - 3s 27ms/step - loss: 0.9939 - accuracy: 0.1810
Epoch 7/100
104/104 [==============================] - 3s 28ms/step - loss: 0.9929 - accuracy: 0.1698
Epoch 8/100
104/104 [==============================] - 3s 27ms/step - loss: 0.9793 - accuracy: 0.1759
Epoch 9/100
104/104 [==============================] - 3s 25ms/step - loss: 0.9863 - accuracy: 0.1956
Epoch 10/100
104/104 [==============================] - 3s 26ms/step - loss: 0.974

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7f33b771fd30>>

In [817]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 60, 50)            23400     
                                                                 
 dropout_4 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_5 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_5 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_6 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_6 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_7 (LSTM)               (None, 50)               

In [29]:
garmin_data.shape

(3383, 52)

In [490]:
window_df = garmin_data.iloc[garmin_data.shape[0] - steps - 1:garmin_data.shape[0] - 1]
input_test = preproc_garmin_data.transform(window_df)

In [491]:
first_prediction = model.predict(np.array([input_test]))

1/1 [==============================] - 0s 51ms/step


In [492]:
first_prediction

array([[ 3.26048732e-01, -1.32820457e-01, -1.08376116e-01,
        -9.08865854e-02,  1.40819505e-01,  3.11233729e-01,
        -2.72869051e-01,  8.03724587e-01,  4.38185900e-01,
        -2.00406015e-02,  5.01094699e-01,  6.59387946e-01,
         3.36818665e-01, -1.06016859e-01,  6.52424574e-01,
        -2.16555190e+00,  3.65880966e-01, -1.04546745e-03,
        -9.07090453e-06,  2.26274505e-03,  1.68244320e-03,
        -1.03137444e-03,  1.87648207e-01,  1.68635771e-01,
         6.59881532e-03,  1.39871584e-02,  2.57512599e-01,
         3.26650262e-01, -5.19587398e-01,  4.69733834e-01,
        -3.37504923e-01]], dtype=float32)

In [493]:
# def initialize_model(input_shape):
#     model = Sequential()
#     model.add(layers.LSTM(64, input_shape=input_shape))
#     model.add(layers.Dense(input_shape[1], activation='softmax'))
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model

In [724]:
y_train[-1]
y_train.shape

(3303, 31)

In [495]:
model.evaluate(np.array([input_test]), np.array([y_train[-1]]))

1/1 [==============================] - 0s 53ms/step - loss: 0.5193 - accuracy: 0.0000e+00


[0.5192509293556213, 0.0]

# Inverse ?

In [754]:
from sklearn.compose import ColumnTransformer

class InvertableColumnTransformer(ColumnTransformer):
    """
    Adds an inverse transform method to the standard sklearn.compose.ColumnTransformer.
    """
    def inverse_transform(self, X):
        if isinstance(X,pd.DataFrame):
            X = X.to_numpy()

        transformer_to_cols = {}
        out = {}
        for t in self.transformers_:
            name = t[0]
            cols = t[-1]
            if name not in ("remainder", "cycle_encoder"):
                transformer_to_cols[name] = cols
                for col in cols:
                    out[col] = []
        for name, indices in self.output_indices_.items():
            transformer = self.named_transformers_.get(name, None)
            start = indices.start + 1 if indices.start != 0 else indices.start
            stop = indices.stop + 1 if indices.start != 0 else indices.stop
            
            arr = X[:, start -1 : stop - 1] if stop > X.shape[1] else X[:, start: stop] 

            if transformer in (None, "remainder", "passthrough", "drop"):
                pass

            else:
                try:
                    if isinstance(transformer, Pipeline):
                        tmp = transformer[1].inverse_transform(arr)
                        current_cols = transformer_to_cols[name]
                        for sub in tmp:
                            for col, val in zip(current_cols, sub):
                                out[col].append(val)
                    else:
                        # print(self.output_indices_)
                        transformer.inverse_transform(arr)
                except Exception as e:

                    print(e)

        return pd.DataFrame.from_dict(out)

In [796]:
def create_preproc_activity_2(activity_df):

    # get numerical data
    data_num = activity_df.select_dtypes(include=np.number)
    data_num

    # get cols numerical
    numerical_features = activity_df.select_dtypes(include=np.number).columns
    numerical_features = set(numerical_features) - set(["avg_stroke_distance",
                                                        "num_active_lengths",
                                                        "num_lengths",
                                                        "max_running_cadence",
                                                        "pool_length",
                                                        "max_cadence",
                                                        "avg_step_length",
                                                        "normalized_power",
                                                        "avg_power",
                                                        "total_strokes",
                                                        "training_stress_score",
                                                        "avg_cadence",
                                                        "max_power"
                                                        ])
    numerical_features = list(numerical_features)

    # Stamina features
    # 205: beginning
    # 206: ending
    # 207: min
    beginning_stamina_features = ["205"]
    numerical_features = list(
        set(numerical_features) - set(beginning_stamina_features))

    # pipeline knn imputer
    pipe_knn_imputer = Pipeline([
        ('knn_imputer', KNNImputer(n_neighbors=4)),
        ('robust_scaler', RobustScaler())
    ])

    # pipeline simple imputer fill value with 100 for beginning stamina (205)
    pipe_nan_to_100 = Pipeline([
        ('simple_imputer', SimpleImputer(strategy="constant", fill_value=100)),
        ('robust_scaler', RobustScaler())
    ])

    # get data and cols categorial feature
    data_cat = activity_df.select_dtypes(exclude=np.number)
    cat_features = data_cat.columns
    cat_features = list(set(cat_features) - set(["pool_length_unit"]))

    # pipeline categorical

    pipe_onehot = Pipeline([
        ("simple_imputer", SimpleImputer(strategy="most_frequent")),
        ("one_hot", OneHotEncoder(sparse_output=False,
         handle_unknown="ignore", drop="if_binary"))
    ])

    full_proces = InvertableColumnTransformer(transformers=[
        ("knn_imputer", pipe_knn_imputer, numerical_features),
        ("cat_encoder", pipe_onehot, ["sport"]),
        ("cycle_encoder", CyclicalFeaturesActivity(),
         ["timestamp", "start_time"]),
        ("imputer 100", pipe_nan_to_100, beginning_stamina_features)
    ]).set_output(transform="pandas")

    return full_proces

In [835]:
preproc = create_preproc_activity(activities)
preproc.fit(activities)
preproc

InvertableColumnTransformer(transformers=[('knn_imputer',
                                           Pipeline(steps=[('knn_imputer',
                                                            KNNImputer(n_neighbors=4)),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['training_load_peak',
                                            'avg_heart_rate', '206',
                                            'max_heart_rate',
                                            'total_anaerobic_training_effect',
                                            'total_calories', '188',
                                            'total_distance',
                                            'enhanced_avg_speed',
                                            'total_training_effect', '178',
                                            'total_ascent...
                                                            SimpleImputer(strategy='most_frequent')),
                                                           ('one_hot',
                                                            OneHotEncoder(drop='if_binary',
                                                                          handle_unknown='ignore',
                                                                          sparse_output=False))]),
                                           ['sport']),
                                          ('cycle_encoder',
                                           CyclicalFeaturesActivity(),
                                           ['timestamp', 'start_time']),
                                          ('imputer 100',
                                           Pipeline(steps=[('simple_imputer',
                                                            SimpleImputer(fill_value=100,
                                                                          strategy='constant')),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['205'])])

In [836]:
first_prediction

array([[ 3.26048732e-01, -1.32820457e-01, -1.08376116e-01,
        -9.08865854e-02,  1.40819505e-01,  3.11233729e-01,
        -2.72869051e-01,  8.03724587e-01,  4.38185900e-01,
        -2.00406015e-02,  5.01094699e-01,  6.59387946e-01,
         3.36818665e-01, -1.06016859e-01,  6.52424574e-01,
        -2.16555190e+00,  3.65880966e-01, -1.04546745e-03,
        -9.07090453e-06,  2.26274505e-03,  1.68244320e-03,
        -1.03137444e-03,  1.87648207e-01,  1.68635771e-01,
         6.59881532e-03,  1.39871584e-02,  2.57512599e-01,
         3.26650262e-01, -5.19587398e-01,  4.69733834e-01,
        -3.37504923e-01]], dtype=float32)

In [840]:
preproc.inverse_transform(first_prediction).iloc[0]

training_load_peak                    69.877296
avg_heart_rate                       117.425385
206                                   74.116966
max_heart_rate                       138.000992
total_anaerobic_training_effect         0.14082
total_calories                       725.802856
188                                    1.454262
total_distance                     21228.177734
enhanced_avg_speed                     4.383346
total_training_effect                  2.143886
178                                  764.599854
total_ascent                          93.531601
enhanced_max_speed                        6.337
207                                   73.815193
total_descent                         87.413361
sport                                   cycling
timestamp                               6:09.36
start_time                              5:14.14
205                                   99.662498
Name: 0, dtype: object

In [838]:
preproc.inverse_transform(np.array([y_train[-10]]))

,training_load_peak,avg_heart_rate,206,max_heart_rate,total_anaerobic_training_effect,total_calories,188,total_distance,enhanced_avg_speed,total_training_effect,178,total_ascent,enhanced_max_speed,207,total_descent,sport,timestamp,start_time,205
0,172.981277,147.0,44.0,164.0,0.0,888.0,3.0,15179.46,4.215,3.8,999.0,44.0,4.945,44.0,42.0,running,11:15.00,10:37.30,99.741181


In [764]:
_sin = 4.69733834e-01
_cos = -3.37504923e-01

In [515]:
np.arccos(_cos) / (2 * np.pi) * 24

7.314995547189589

In [516]:
np.arcsin(_sin) / (2 * np.pi) * 24

1.8678013675153158

In [765]:
import math
angle2 = math.atan2(_sin, _cos)  # ALWAYS USE THIS
angle2 *= 180 / math.pi
if angle2 < 0: angle2 += 360
print(angle2/24)

5.23738912903143


In [778]:
def convert_sin_cos_to_hour(s, c):
    angle = math.atan2(s, c)  # ALWAYS USE THIS
    angle *= 180 / math.pi
    if angle < 0: angle += 360
    print(angle / 24)

In [779]:

convert_sin_cos_to_hour(-1.00000000e+00, -1.83697020e-16 )

11.25


In [780]:
convert_sin_cos_to_hour(-9.65925826e-01, -2.58819045e-01)

10.625000000057799


In [772]:
   # 0.46973383 -0.33750492
convert_sin_cos_to_hour(0.32665026, -0.5195874)

6.160148252132713


In [773]:
convert_sin_cos_to_hour(0.46973383, -0.33750492)

5.2373891286091245


# Test From python code

In [23]:
create_train_and_save_model()

⌛ Loading activities data from BigQuery server... ⌛ 
✅ activities data loaded, with shape (3363, 35)
⌛ Loading garmin_data data from BigQuery server... ⌛ 
✅ garmin_data data loaded, with shape (3445, 53)


⌛ Creating Sliding Window dataset... ⌛: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 3303/3303 [04:48<00:00, 11.44it/s]


Epoch 1/200
207/207 [==============================] - 10s 25ms/step - loss: 1.0738 - accuracy: 0.1211
Epoch 2/200
207/207 [==============================] - 6s 28ms/step - loss: 1.0409 - accuracy: 0.1514
Epoch 3/200
207/207 [==============================] - 6s 30ms/step - loss: 1.0199 - accuracy: 0.1571
Epoch 4/200
207/207 [==============================] - 6s 29ms/step - loss: 1.0167 - accuracy: 0.1741
Epoch 5/200
207/207 [==============================] - 6s 29ms/step - loss: 0.9930 - accuracy: 0.1817
Epoch 6/200
207/207 [==============================] - 6s 29ms/step - loss: 0.9978 - accuracy: 0.1826
Epoch 7/200
207/207 [==============================] - 6s 28ms/step - loss: 0.9772 - accuracy: 0.1904
Epoch 8/200
207/207 [==============================] - 6s 28ms/step - loss: 0.9680 - accuracy: 0.1783
Epoch 9/200
207/207 [==============================] - 6s 28ms/step - loss: 0.9702 - accuracy: 0.1962
Epoch 10/200
207/207 [==============================] - 6s 28ms/step - loss: 0.96

# Predict for given date

In [9]:
!pwd

/home/nico/code/nicblanc/overcome-tomorrow/notebooks


In [1]:
preproc_garmin_data, preproc_activity, model = load_preprocessors_and_model(model_path="../raw_data", preprocessors_path="../raw_data")

NameError: name 'load_preprocessors_and_model' is not defined

In [3]:
garmin_data, activities = get_data()

⌛ Loading activities data from BigQuery server... ⌛ 
✅ activities data loaded, with shape (3363, 35)
⌛ Loading garmin_data data from BigQuery server... ⌛ 
✅ garmin_data data loaded, with shape (3445, 53)


In [4]:
models_dict = get_all_models()

✅ Latest model downloaded from cloud storage
✅ Latest preprocessors downloaded from cloud storage


2024-03-24 18:34:56.630505: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-24 18:34:56.635262: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-24 18:34:56.635336: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-24 18:34:56.635603: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [7]:

# model_dict = models_dict[pathlib.PurePath(MODEL_NAME).stem]
model_dict = models_dict["first_model"]
model = model_dict[MODEL_KEY]
preproc_garmin_data = model_dict[PREPROC_GARMIN_DATA_KEY]
preproc_activity = model_dict[PREPROC_ACTIVITY_KEY]

In [9]:
from datetime import datetime, timedelta
now = datetime.now()
delta = timedelta(days = 30)
date = (now - delta)#.strftime('%Y-%m-%d %H:%M:%S')
date

datetime.datetime(2024, 2, 23, 18, 37, 37, 981284)

In [10]:
get_sliding_window_for_date(garmin_data, date).iloc[-1]["beginTimestamp"]

Timestamp('2024-02-23 05:02:42+0000', tz='UTC')

In [11]:
get_sliding_window_for_date(garmin_data, date).tail(3)

,calendarDate,start_sleep,end_sleep,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,qualityScore,beginTimestamp,activityTrainingLoad,activityType,aerobicTrainingEffect,aerobicTrainingEffectMessage,anaerobicTrainingEffect,anaerobicTrainingEffectMessage,avgBikeCadence,avgHr,avgPower,avgRunCadence,avgSpeed,calories,caloriesConsumed,distance,duration,maxHr,maxPower,maxRunCadence,maxSpeed,moderateIntensityMinutes,normPower,sportType,trainingEffectLabel,trainingStressScore,vigorousIntensityMinutes,totalKilocalories,activeKilocalories,totalDistanceMeters,wellnessDistanceMeters,highlyActiveSeconds,activeSeconds,floorsAscendedInMeters,floorsDescendedInMeters,restingHeartRate,minAvgHeartRate,maxAvgHeartRate,hydration_sweatLossInML,respiration_avgWakingRespirationValue,respiration_highestRespirationValue,respiration_lowestRespirationValue,bodyBattery_Highest,bodyBattery_Lowest,allDayStress_awake,allDayStress_asleep
3307,2024-02-22,2024-02-21 20:05:00+00:00,2024-02-22 02:41:15+00:00,4380.0,14640.0,4740.0,0.0,91.0,2024-02-22 05:09:15+00:00,129.212799,virtual_ride,3.4,IMPACTING_TEMPO_22,0.0,NO_ANAEROBIC_BENEFIT_0,74.0,124.0,220.0,NaN,0.9039,7919.13780,NaN,7.051845e+06,7.801221e+06,147.0,381.0,NaN,1.3753,4.0,221.0,CYCLING,TEMPO,136.899994,89.0,3789.0,1851.0,79421.0,8941.0,872.0,5311.0,80.407,84.068,42.0,39.0,144.0,1894.0,13.0,18.0,9.0,95.0,35.0,20.0,14.0
3308,2024-02-22,2024-02-21 20:05:00+00:00,2024-02-22 02:41:15+00:00,4380.0,14640.0,4740.0,0.0,91.0,2024-02-22 18:06:30+00:00,5.405563,walking,0.3,NO_AEROBIC_BENEFIT_18,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,92.0,NaN,56.0,0.1425,909.23434,NaN,4.152450e+05,2.914775e+06,108.0,NaN,60.0,0.2230,44.0,NaN,STEPS,UNKNOWN,NaN,0.0,3789.0,1851.0,79421.0,8941.0,872.0,5311.0,80.407,84.068,42.0,39.0,144.0,1894.0,13.0,18.0,9.0,95.0,35.0,20.0,14.0
3309,2024-02-23,2024-02-22 20:15:00+00:00,2024-02-23 02:46:00+00:00,6420.0,11400.0,5640.0,0.0,93.0,2024-02-23 05:02:42+00:00,0.229233,strength_training,0.0,NO_AEROBIC_BENEFIT_18,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,66.0,NaN,NaN,0.0000,54.47026,NaN,0.000000e+00,4.020890e+05,80.0,NaN,NaN,NaN,NaN,NaN,TRAINING,UNKNOWN,NaN,NaN,3442.0,1508.0,27649.0,27649.0,8011.0,4203.0,142.318,86.795,41.0,36.0,151.0,1783.0,13.0,21.0,10.0,87.0,40.0,20.0,14.0


In [12]:
predict_for_date(garmin_data, preproc_garmin_data, preproc_activity, model, date)

2024-03-24 18:37:42.712849: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


1/1 [==============================] - 4s 4s/step


2024-03-24 18:37:44.423014: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


,enhanced_avg_speed,207,max_heart_rate,total_training_effect,total_descent,total_anaerobic_training_effect,total_ascent,188,avg_heart_rate,training_load_peak,178,enhanced_max_speed,total_calories,206,total_distance,sport,timestamp,start_time,205
0,2.553987,66.523117,145.054398,2.775534,75.358734,-0.279277,58.247414,2.141792,146.414734,95.08049,572.407837,5.600417,109.511475,66.157188,6180.155273,walking,10:58:08,9:07:19,99.320198


In [13]:
predict_for_last_n_days(garmin_data, preproc_garmin_data, preproc_activity, model, 30)

1/1 [==============================] - 0s 28ms/step


,enhanced_avg_speed,207,max_heart_rate,total_training_effect,total_descent,total_anaerobic_training_effect,total_ascent,188,avg_heart_rate,training_load_peak,178,enhanced_max_speed,total_calories,206,total_distance,sport,timestamp,start_time,205
0,6.933899,76.877144,157.072052,4.976038,196.454376,0.326005,-9.429638,4.661386,212.279053,214.096405,911.542725,-0.336910,595.550598,79.438141,19383.794922,cycling,10:28:37,8:39:10,96.398323
1,5.763358,74.532639,153.493927,4.302052,159.286697,0.190431,-2.906673,3.984016,192.779953,178.625900,844.166016,0.323214,513.192200,77.042236,16853.410156,cycling,10:37:11,8:51:36,94.995750
2,5.361950,73.642365,151.995514,4.046438,141.661224,0.118136,15.032663,3.515321,181.835175,159.797653,763.317993,1.862635,436.663971,74.871437,13557.402344,cycling,10:28:55,8:40:58,97.030357
3,3.730323,69.659653,148.597794,3.222520,95.576782,-0.109191,39.622021,2.655646,157.292450,115.417755,645.488159,3.997140,276.877808,70.692810,9009.079102,cycling,11:18:14,9:36:42,99.358543
4,2.780043,67.797134,145.810562,2.615094,68.095284,-0.213398,51.617886,2.179473,142.791336,86.415161,602.361633,5.062602,248.062805,70.065437,7639.898438,walking,12:02:59,10:51:32,99.538277
5,2.408457,68.008553,144.559235,2.199656,49.028709,-0.184652,41.930859,1.819806,133.705231,74.388840,695.182983,4.285641,338.092834,72.470398,11471.712891,cycling,11:31:19,10:41:58,93.497200
6,3.179313,69.118629,146.034698,2.821287,82.912384,-0.133987,47.846519,2.333947,149.888565,102.983551,665.881958,4.754833,306.901611,71.548965,10243.996094,cycling,10:50:55,9:36:23,97.041962
7,3.445473,69.204750,146.822617,3.240098,115.542488,-0.138257,38.760189,2.658771,167.417053,126.641571,682.803711,3.896699,241.245468,69.481537,11464.425781,cycling,10:56:31,9:08:08,98.070198
8,5.740035,74.025215,149.997665,4.853916,207.445496,0.140743,-7.900784,4.037432,216.110229,221.072174,901.040527,-0.091968,364.974609,72.158295,20667.382812,cycling,8:15:17,6:16:10,90.591072
9,3.608934,71.659851,146.417252,2.058266,52.576645,0.058014,-68.562302,2.259534,135.480377,69.296402,825.133179,-5.077532,604.714355,77.851326,17107.751953,cycling,10:34:14,9:59:43,65.760307


In [4]:
garmin_data, activities = get_data()

⌛ Loading activities data from BigQuery server... ⌛ 
✅ activities data loaded, with shape (3363, 35)
⌛ Loading garmin_data data from BigQuery server... ⌛ 
✅ garmin_data data loaded, with shape (3445, 53)


In [19]:
preproc_activity.inverse_transform(preproc_activity.transform(activities))["enhanced_max_speed"].describe()

count    3363.000000
mean        5.339040
std         5.160462
min         0.000000
25%         1.726000
50%         3.760000
75%         9.377000
max        65.399000
Name: enhanced_max_speed, dtype: float64

In [169]:
garmin_data.tail()

,calendarDate,start_sleep,end_sleep,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,qualityScore,beginTimestamp,activityTrainingLoad,activityType,aerobicTrainingEffect,aerobicTrainingEffectMessage,anaerobicTrainingEffect,anaerobicTrainingEffectMessage,avgBikeCadence,avgHr,avgPower,avgRunCadence,avgSpeed,calories,caloriesConsumed,distance,duration,maxHr,maxPower,maxRunCadence,maxSpeed,moderateIntensityMinutes,normPower,sportType,trainingEffectLabel,trainingStressScore,vigorousIntensityMinutes,totalKilocalories,activeKilocalories,totalDistanceMeters,wellnessDistanceMeters,highlyActiveSeconds,activeSeconds,floorsAscendedInMeters,floorsDescendedInMeters,restingHeartRate,minAvgHeartRate,maxAvgHeartRate,hydration_sweatLossInML,respiration_avgWakingRespirationValue,respiration_highestRespirationValue,respiration_lowestRespirationValue,bodyBattery_Highest,bodyBattery_Lowest,allDayStress_awake,allDayStress_asleep
3378,2024-03-14,2024-03-13 21:05:00+00:00,2024-03-14 03:20:00+00:00,3420.0,15420.0,3660.0,0.0,79.0,2024-03-14 11:25:57+00:00,0.686172,strength_training,0.0,NO_AEROBIC_BENEFIT_18,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,75.0,NaN,NaN,0.0000,87.99042,NaN,0.000000e+00,4.022960e+05,89.0,NaN,NaN,NaN,NaN,NaN,TRAINING,UNKNOWN,NaN,NaN,4563.0,2622.0,87184.0,16435.0,5810.0,2023.0,84.226,60.632,40.0,35.0,149.0,2139.0,13.0,20.0,7.0,62.0,26.0,27.0,21.0
3379,2024-03-14,2024-03-13 21:05:00+00:00,2024-03-14 03:20:00+00:00,3420.0,15420.0,3660.0,0.0,79.0,2024-03-14 17:21:40+00:00,116.148071,running,3.3,IMPROVING_AEROBIC_BASE_8,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,142.0,338.0,84.0,0.3955,3892.52858,NaN,1.601268e+06,4.048596e+06,153.0,413.0,90.0,0.4283,0.0,339.0,RUNNING,AEROBIC_BASE,NaN,68.0,4563.0,2622.0,87184.0,16435.0,5810.0,2023.0,84.226,60.632,40.0,35.0,149.0,2139.0,13.0,20.0,7.0,62.0,26.0,27.0,21.0
3380,2024-03-15,2024-03-14 20:07:00+00:00,2024-03-15 02:47:53+00:00,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 05:25:09+00:00,144.809784,gravel_cycling,3.5,IMPACTING_TEMPO_22,0.0,NO_ANAEROBIC_BENEFIT_0,77.0,127.0,218.0,NaN,0.8657,8459.65038,NaN,7.261684e+06,8.388064e+06,142.0,418.0,NaN,1.1915,1.0,221.0,CYCLING,TEMPO,147.399994,129.0,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0
3381,2024-03-15,2024-03-14 20:07:00+00:00,2024-03-15 02:47:53+00:00,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 10:57:38+00:00,0.604935,strength_training,0.0,NO_AEROBIC_BENEFIT_18,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,75.0,NaN,NaN,0.0000,79.61038,NaN,0.000000e+00,4.022590e+05,90.0,NaN,NaN,NaN,NaN,NaN,FITNESS_EQUIPMENT,UNKNOWN,NaN,NaN,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0
3382,2024-03-15,2024-03-14 20:07:00+00:00,2024-03-15 02:47:53+00:00,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 11:14:42+00:00,22.125320,running,1.5,RECOVERY_5,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,126.0,319.0,84.0,0.3521,586.60280,NaN,2.337730e+05,6.639260e+05,139.0,442.0,88.0,0.4749,0.0,323.0,RUNNING,RECOVERY,NaN,10.0,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0


In [170]:
activities.tail()

,timestamp,total_anaerobic_training_effect,enhanced_avg_speed,avg_stroke_distance,total_training_effect,num_active_lengths,max_running_cadence,max_heart_rate,sub_sport,total_descent,start_time,total_distance,pool_length,178,total_calories,max_cadence,sport,188,avg_step_length,enhanced_max_speed,205,num_lengths,206,207,pool_length_unit,normalized_power,training_load_peak,total_ascent,avg_power,total_strokes,training_stress_score,avg_cadence,avg_heart_rate,max_power,activity_id
3358,2024-03-14 11:32:43+00:00,0.0,0.000,NaN,0.0,NaN,NaN,89.0,strength_training,NaN,2024-03-14 11:25:57+00:00,0.00,NaN,53.0,21.0,NaN,training,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.686172,NaN,NaN,NaN,NaN,NaN,75.0,NaN,nicko64@hotmail.fr_241188067403
3359,2024-03-14 18:32:31+00:00,0.0,3.955,NaN,3.3,NaN,90.0,153.0,generic,52.0,2024-03-14 17:21:40+00:00,16012.68,NaN,1124.0,929.0,90.0,running,2.0,1404.1,4.283,94.0,NaN,71.0,71.0,None,339.0,116.148071,56.0,338.0,NaN,NaN,84.0,142.0,413.0,nicko64@hotmail.fr_241246052903
3360,2024-03-15 07:45:23+00:00,0.0,8.657,NaN,3.5,NaN,NaN,142.0,gravel_cycling,298.0,2024-03-15 05:25:09+00:00,72616.84,NaN,1371.0,2019.0,104.0,cycling,3.0,NaN,11.915,95.0,NaN,67.0,67.0,None,221.0,144.809784,296.0,218.0,10737.0,147.4,77.0,127.0,418.0,nicko64@hotmail.fr_241349530713
3361,2024-03-15 11:05:33+00:00,0.0,0.000,NaN,0.0,NaN,NaN,90.0,strength_training,NaN,2024-03-15 10:57:38+00:00,0.00,NaN,39.0,19.0,NaN,training,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.604935,NaN,NaN,NaN,NaN,NaN,75.0,NaN,nicko64@hotmail.fr_241374800667
3362,2024-03-15 11:26:17+00:00,0.0,3.521,NaN,1.5,NaN,88.0,139.0,generic,13.0,2024-03-15 11:14:42+00:00,2337.73,NaN,184.0,140.0,88.0,running,1.0,1265.6,4.749,95.0,NaN,91.0,91.0,None,323.0,22.125320,16.0,319.0,NaN,NaN,84.0,126.0,442.0,nicko64@hotmail.fr_241377639543


In [22]:
prediction = predict_for_date(garmin_data, preproc_garmin_data, preproc_activity, model, datetime(2024, 3, 15, 0, 0, 0))
prediction

1/1 [==============================] - 0s 25ms/step


,max_heart_rate,total_distance,total_training_effect,training_load_peak,188,178,206,total_descent,total_ascent,avg_heart_rate,total_calories,enhanced_avg_speed,total_anaerobic_training_effect,207,enhanced_max_speed,sport,timestamp,start_time,205
0,160.865479,2138.778809,1.875135,18.741255,2.34506,782.651978,72.007187,133.633362,79.812675,124.053123,976.130371,8.270152,0.403994,73.477829,12.729819,cycling,9:47:46,8:06:27,98.45932


In [7]:
get_sliding_window_for_date(garmin_data, datetime(2024, 3, 15, 0, 0, 0)).tail(3)

,calendarDate,start_sleep,end_sleep,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,qualityScore,beginTimestamp,activityTrainingLoad,activityType,aerobicTrainingEffect,aerobicTrainingEffectMessage,anaerobicTrainingEffect,anaerobicTrainingEffectMessage,avgBikeCadence,avgHr,avgPower,avgRunCadence,avgSpeed,calories,caloriesConsumed,distance,duration,maxHr,maxPower,maxRunCadence,maxSpeed,moderateIntensityMinutes,normPower,sportType,trainingEffectLabel,trainingStressScore,vigorousIntensityMinutes,totalKilocalories,activeKilocalories,totalDistanceMeters,wellnessDistanceMeters,highlyActiveSeconds,activeSeconds,floorsAscendedInMeters,floorsDescendedInMeters,restingHeartRate,minAvgHeartRate,maxAvgHeartRate,hydration_sweatLossInML,respiration_avgWakingRespirationValue,respiration_highestRespirationValue,respiration_lowestRespirationValue,bodyBattery_Highest,bodyBattery_Lowest,allDayStress_awake,allDayStress_asleep
3376,2024-03-13,2024-03-12 20:02:00+00:00,2024-03-13 03:57:00+00:00,2520.0,22200.0,3360.0,420.0,73.0,2024-03-13 17:43:45+00:00,125.475555,running,3.3,IMPROVING_AEROBIC_BASE_8,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,141.0,338.0,84.0,0.4022,3951.18886,NaN,1.634100e+06,4.063275e+06,159.0,431.0,91.0,0.4721,0.0,340.0,RUNNING,AEROBIC_BASE,NaN,66.0,4509.0,2576.0,87474.0,16843.0,6225.0,1744.0,86.614,58.163,39.0,38.0,150.0,2382.0,13.0,19.0,9.0,61.0,24.0,28.0,24.0
3377,2024-03-14,2024-03-13 21:05:00+00:00,2024-03-14 03:20:00+00:00,3420.0,15420.0,3660.0,0.0,79.0,2024-03-14 05:39:29+00:00,164.412140,gravel_cycling,3.7,IMPACTING_TEMPO_22,0.0,NO_ANAEROBIC_BENEFIT_0,75.0,129.0,223.0,NaN,0.8895,8183.10906,NaN,7.076639e+06,7.956113e+06,138.0,379.0,NaN,1.1785,1.0,226.0,CYCLING,TEMPO,145.199997,121.0,4563.0,2622.0,87184.0,16435.0,5810.0,2023.0,84.226,60.632,40.0,35.0,149.0,2139.0,13.0,20.0,7.0,62.0,26.0,27.0,21.0
3378,2024-03-14,2024-03-13 21:05:00+00:00,2024-03-14 03:20:00+00:00,3420.0,15420.0,3660.0,0.0,79.0,2024-03-14 11:25:57+00:00,0.686172,strength_training,0.0,NO_AEROBIC_BENEFIT_18,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,75.0,NaN,NaN,0.0000,87.99042,NaN,0.000000e+00,4.022960e+05,89.0,NaN,NaN,NaN,NaN,NaN,TRAINING,UNKNOWN,NaN,NaN,4563.0,2622.0,87184.0,16435.0,5810.0,2023.0,84.226,60.632,40.0,35.0,149.0,2139.0,13.0,20.0,7.0,62.0,26.0,27.0,21.0


In [44]:
real = activities[activities["start_time"].dt.strftime('%Y-%m-%d %H:%M:%S') >= datetime(2024, 3, 15, 0, 0, 0).strftime('%Y-%m-%d %H:%M:%S')].iloc[[0]]
preproc_activity.inverse_transform(preproc_activity.transform(real))
# activities[activities["start_time"].dt.strftime('%Y-%m-%d %H:%M:%S') >= datetime(2024, 3, 15, 0, 0, 0).strftime('%Y-%m-%d %H:%M:%S')]

,max_heart_rate,total_distance,total_training_effect,training_load_peak,188,178,206,total_descent,total_ascent,avg_heart_rate,total_calories,enhanced_avg_speed,total_anaerobic_training_effect,207,enhanced_max_speed,sport,timestamp,start_time,205
0,142.0,72616.84,3.5,144.809784,3.0,1371.0,67.0,298.0,296.0,127.0,2019.0,8.657,0.0,67.0,11.915,cycling,7:45:00,5:25:00,95.0


In [51]:
pd.concat([real, prediction],  keys=['real', 'prediction']).info()


,,max_heart_rate,total_distance,total_training_effect,training_load_peak,188,178,206,total_descent,total_ascent,avg_heart_rate,total_calories,enhanced_avg_speed,total_anaerobic_training_effect,207,enhanced_max_speed,sport,timestamp,start_time,205
prediction,0,160.865479,2138.778809,1.875135,18.741255,2.34506,782.651978,72.007187,133.633362,79.812675,124.053123,976.130371,8.270152,0.403994,73.477829,12.729819,cycling,9:47:46,8:06:27,98.45932


In [148]:
s, c = convert_time_to_sin_cos(pd.to_datetime(real["timestamp"]))

In [149]:
s, c

(3360    0.896873
 Name: timestamp, dtype: float64,
 3360   -0.442289
 Name: timestamp, dtype: float64)

In [157]:
convert_sin_cos_to_hour(s, c)

'7:45:00'

In [139]:
angle = math.atan2(s, c)
angle *= 180.0 / math.pi
if angle < 0:
    angle += 360.0

In [140]:
angle

116.25

In [138]:
time = angle * 24.0 / 360.0
time

7.75

In [137]:
hours = int(time)
hours

7

In [133]:
(time*60) % 60

45.0

In [134]:
(time*3600) % 60

0.0


# Test predict VS real

In [22]:
predict_vs_real_for_date(garmin_data, activities, preproc_garmin_data, preproc_activity, model, datetime(2022, 12, 17, 4, 0, 0))

1/1 [==============================] - 0s 26ms/step


,,enhanced_avg_speed,207,max_heart_rate,total_training_effect,total_descent,total_anaerobic_training_effect,total_ascent,188,avg_heart_rate,training_load_peak,178,enhanced_max_speed,total_calories,206,total_distance,sport,timestamp,start_time,205,avg_stroke_distance,num_active_lengths,max_running_cadence,sub_sport,pool_length,max_cadence,avg_step_length,num_lengths,pool_length_unit,normalized_power,avg_power,total_strokes,training_stress_score,avg_cadence,max_power,activity_id
prediction,0,8.385546,79.874832,161.758652,5.895829,246.684235,0.493744,-38.416122,5.235639,243.096207,265.144196,1030.460449,-2.944492,706.091736,82.259895,24156.673828,cycling,10:56:20,8:44:32,97.314308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
real,2333,0.554000,NaN,124.000000,0.700000,NaN,0.000000,NaN,0.000000,108.000000,9.603165,NaN,0.607000,200.000000,NaN,1000.000000,swimming,2022-12-17 08:45:36+00:00,2022-12-17 08:14:54+00:00,NaN,0.0,40.0,NaN,lap_swimming,25.0,NaN,NaN,40.0,metric,NaN,NaN,632.0,NaN,21.0,NaN,nicko64@hotmail.fr_166311365922


In [147]:
predict_vs_real_for_last_n_days(garmin_data, activities, preproc_garmin_data, preproc_activity, model, 5)

1/1 [==============================] - 0s 30ms/step


max_heart_rate  total_distance  total_training_effect  \
predictions 0         166.394073     8890.718750               0.272615   
            1         151.727814      -31.257246               2.307117   
            2         163.754379     3777.704346               1.609562   
            3         164.461731     4369.923828               1.352545   
            4         161.543304     3671.725098               2.050171   
reals       3346      160.000000    33170.150000               4.400000   
            3347       85.000000        0.000000               0.000000   
            3348      121.000000    70089.590000               3.000000   
            3349      155.000000    10276.660000               3.100000   
            3350      106.000000        0.000000               0.000000   

                  training_load_peak       188          178        206  \
predictions 0              58.751450  2.516154   752.943420  74.848923   
            1               6.914112  2.248977   622.230652  69.139931   
            2              28.205944  2.423476   796.056519  72.895424   
            3              31.305290  2.458719   801.838074  73.185478   
            4              27.918797  2.349540   724.815918  72.586151   
reals       3346          245.418472  2.000000  2087.000000  47.000000   
            3347            0.411163  0.000000    47.000000        NaN   
            3348           85.867844  2.000000  1362.000000  77.000000   
            3349           97.694046  2.000000   689.000000  50.000000   
            3350            0.515244  0.000000    49.000000        NaN   

                  total_descent  total_ascent  avg_heart_rate  total_calories  \
predictions 0        132.864288    108.427322      134.867310      934.579956   
            1         93.590958     62.742508      116.003830      730.828491   
            2        139.640015     86.953461      127.252090      989.843262   
            3        139.134094     89.873749      128.658859      997.409424   
            4        112.209572     72.648987      125.083389      843.500610   
reals       3346      89.000000     92.000000      140.000000     1975.000000   
            3347            NaN           NaN       67.000000       16.000000   
            3348     362.000000    147.000000      108.000000     1732.000000   
            3349      31.000000     32.000000      138.000000      596.000000   
            3350            NaN           NaN       70.000000       18.000000   

                  enhanced_avg_speed  total_anaerobic_training_effect  \
predictions 0               8.258449                         0.763744   
            1               5.649906                         0.186064   
            2               8.602686                         0.499410   
            3               8.621147                         0.532708   
            4               6.604935                         0.310152   
reals       3346            4.012000                         0.000000   
            3347            0.000000                         0.000000   
            3348            8.954000                         0.000000   
            3349            3.999000                         0.000000   
            3350            0.000000                         0.000000   

                        207  enhanced_max_speed     sport  \
predictions 0     63.543224           12.801803   cycling   
            1     76.299057            9.076134   cycling   
            2     71.768967           13.282360   cycling   
            3     70.108719           13.020673   cycling   
            4     74.567436           10.630401   cycling   
reals       3346  47.000000            4.441000   running   
            3347        NaN                 NaN  training   
            3348  75.000000           14.444000   cycling   
            3349  50.000000            4.507000   running   
            3350        NaN                 NaN  training   

         